In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
PATH = '../data/SocialTalk/predict'
SEPARATOR = ','

In [3]:
categorical_df = pd.read_csv(f'{PATH}/categorical_buckets.csv', sep=SEPARATOR, low_memory=False, index_col=0).fillna('Unknown')

In [4]:
age_df = pd.read_csv(f'{PATH}/age_categories.csv', sep=SEPARATOR, low_memory=False)
audience_df = pd.read_csv(f'{PATH}/audience_categories.csv', sep=SEPARATOR, low_memory=False)
interest_df = pd.read_csv(f'{PATH}/interest_categories.csv', sep=SEPARATOR, low_memory=False)

In [5]:
for column in categorical_df.columns:
    categorical_df[column] = categorical_df[column].apply(lambda x: f'{column}/{x}')

In [6]:
x = categorical_df.unstack().reset_index()
x.columns = ['rel', 'head', 'tail']

x['head'] = x['head'].apply(lambda x: f'entity/{x}')
x['rel'] = x['rel'].apply(lambda x: f'relation/{x}')

categorical_triples = x[['head', 'rel', 'tail']]

In [7]:
RELATION_NAME = 'age'
x = age_df
x.columns = ['tail', 'head']
x['rel'] = f'relation/{RELATION_NAME}'

x['head'] = x['head'].apply(lambda x: f'entity/{x}')
x['tail'] = x['tail'].apply(lambda x: f'{RELATION_NAME}/{x}')

age_triples = x[['head', 'rel', 'tail']]

In [8]:
RELATION_NAME = 'audience'
x = audience_df
x.columns = ['head', 'tail']
x['rel'] = f'relation/{RELATION_NAME}'

x['head'] = x['head'].apply(lambda x: f'entity/{x}')
x['tail'] = x['tail'].apply(lambda x: f'{RELATION_NAME}/{x}')

audience_triples = x[['head', 'rel', 'tail']]

In [9]:
RELATION_NAME = 'interest'
x = interest_df
x.columns = ['head', 'tail']
x['rel'] = f'relation/{RELATION_NAME}'

x['head'] = x['head'].apply(lambda x: f'entity/{x}')
x['tail'] = x['tail'].apply(lambda x: f'{RELATION_NAME}/{x}')

interest_triples = x[['head', 'rel', 'tail']]

In [10]:
triples = pd.concat([categorical_triples, age_triples, audience_triples, interest_triples])

In [11]:
triples

,head,rel,tail
0,entity/nunomarkl,relation/Social network,Social network/Instagram
1,entity/vanesssamartins,relation/Social network,Social network/Instagram
2,entity/acatmoreira,relation/Social network,Social network/Instagram
3,entity/dulceida,relation/Social network,Social network/TikTok
4,entity/mariapombo,relation/Social network,Social network/Instagram
...,...,...,...
75,entity/martabaceiredo,relation/interest,interest/Carrefour
76,entity/dulceida,relation/interest,interest/ASOS
77,entity/_raquelsampaio_,relation/interest,interest/Clinique
78,entity/jessicavferreirasilva,relation/interest,interest/Bershka


In [12]:
triples.to_csv('../data/SocialTalk/PredictionTriples.csv', index=None)

In [11]:
BREAK

NameError: name 'BREAK' is not defined

#### PyKeen specifics

In [ ]:
sampled_entities = triples['head'].sample(2000).tolist()
sampled_entities += [f'entity/{i}' for i in range(650)]
sampled_entities = set(sampled_entities)

In [ ]:
sampled_triples = triples[triples['head'].isin(sampled_entities)]

In [ ]:
from pykeen.triples import TriplesFactory

tf = TriplesFactory.from_labeled_triples(sampled_triples.values)
trans_train, ind_train, ind_test, ind_val = tf.split([.4, .3, .2, .1])

In [ ]:
BASE_URL = '../data/SocialTalk'

np.savetxt(f'{BASE_URL}/training/train.txt', trans_train.triples, fmt='%s', delimiter='\t')
np.savetxt(f'{BASE_URL}/inference/train.txt', ind_train.triples, fmt='%s', delimiter='\t')
np.savetxt(f'{BASE_URL}/inference/valid.txt', ind_test.triples, fmt='%s', delimiter='\t')
np.savetxt(f'{BASE_URL}/inference/test.txt', ind_val.triples, fmt='%s', delimiter='\t')